# DuckDB
> SQL em pandas e comparações (dask, datatable)

- toc: true 
- badges: true
- comments: true
- categories: [demo, sql, pandas, duckdb, dask, datatable, polars, tabelas, benchmarks]
- image: https://images.unsplash.com/photo-1526716121440-dc3b4f254a0a?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=700&q=80

# Introdução

Em machine learning, a gente sempre comenta como nós precisamos de dados. E precisamos de dados bem curados. Talvez você até já tenha ouvido a frase: "garbage in, garbage out", né?

Na vida real, isso é bem verdade. E gastar mais tempo criando novas features, ou limpando os dados acaba muitas vezes impactando os resultados dos modelos mais que qualquer outra coisa. Não é à toa que grande parte dos ganhadores das competições do [Kaggle](https://www.kaggle.com/) usam os mesmo modelos - [ensembles](https://towardsdatascience.com/xgboost-lightgbm-and-other-kaggle-competition-favorites-6212e8b0e835). E, na maior parte, quem ganha são os que tem os melhores dados.

Então, antes de mexermos em modelos, temos que manipular os dados. Mas então como manipular esses dados?

## Pandas

A resposta mais comum é [pandas](https://pandas.pydata.org/)! Se você já mexeu com dados, provavelmente já ouviu falar sobre essa biblioteca. Ela é, de longe, a mais popular pra manipulação de dados tabulares. Ela foi construída em cima de [NumPy](https://numpy.org/), que usa C++ por trás dos panos.

> Em Python temos o GIL que limita o uso de threads pra evitar que mais de um thread tente escrever/apagar as mesmas partes da memória ao mesmo tempo ([condição de corrida](https://pt.wikipedia.org/wiki/Condi%C3%A7%C3%A3o_de_corrida)). Quando usamos Numpy e chegamos no nível de C, o programa não está sujeito ao GIL e então podemos parallelizar as coisas.

Mas se você já ouviu falar de pandas, é capaz que não tenha ouvido coisas boas. Especialmente quem vem da linguagem de programação em R usando o [dplyr](https://dplyr.tidyverse.org/) tem o que falar 😅. O número de perguntas no [StackOverflow](https://stackoverflow.com/) relacionadas à pandas cresceu bastante durante os anos. Parte pelo aumento em popularidade da biblioteca, mas parte pela dificuldade da API do pandas.

![](2021-06-13-pandas_and_alternatives/stackoverflow_questions.png)

Além disso, existem vários pontos em que podemos otimizar a performance de pandas. Isso pelo fato da biblioteca ter sido criada em cima de uma outra. Vamos dar uma olhada na performance de pandas.

### Dados

Vamos usar o TPC-H dataset, focando no `lineitem` e `orders`, que são as maiores tabelas que somam 1GB. Para medir o tempo de execução, vamos unir as duas tabelas, filtram as linhas, agrupam os dados e computamos alguns dados agregados (máximo, mínimo, etc.). O que estamos fazendo em si não é muito importante, mas é um tipo de transformação que encontraríamos na vida real.

Vamos tentar otimizar tudo ao máximo, e vamos focar na transformação dos dados

- Vamos medir o tempo da transformação (excluindo o tempo de ler os dados)
- Vamos filtrar o quanto antes
- Vamos encadear operações

In [3]:
# hide
!pip install duckdb pandas 'dask[dataframe]' datatable polars pandasql > /dev/null
!wget -q https://github.com/cwida/duckdb-data/releases/download/v1.0/lineitemsf1.snappy.parquet
!wget -q https://github.com/cwida/duckdb-data/releases/download/v1.0/orders.parquet

In [7]:
# hide
import pandas as pd
import dask.dataframe as dd
import datatable as dt
import duckdb
import polars as pl
from datatable import by, f, g, join

In [10]:
#hide
lineitem = pd.read_parquet("lineitemsf1.snappy.parquet")
orders = pd.read_parquet("orders.parquet")

,o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment
0,1,36901,O,173665.47,1996-01-02,5-LOW,Clerk#000000951,0,nstructions sleep furiously among
1,2,78002,O,46929.18,1996-12-01,1-URGENT,Clerk#000000880,0,"foxes. pending accounts at the pending, silen..."
2,3,123314,F,193846.25,1993-10-14,5-LOW,Clerk#000000955,0,sly final accounts boost. carefully regular id...
3,4,136777,O,32151.78,1995-10-11,5-LOW,Clerk#000000124,0,"sits. slyly regular warthogs cajole. regular, ..."
4,5,44485,F,144659.20,1994-07-30,5-LOW,Clerk#000000925,0,quickly. bold deposits sleep slyly. packages u...
...,...,...,...,...,...,...,...,...,...
1499995,5999972,143594,O,114856.68,1996-05-02,3-MEDIUM,Clerk#000000536,0,y express accounts above the blithely bold
1499996,5999973,32071,O,68906.56,1997-07-13,4-NOT SPECIFIED,Clerk#000000130,0,special ideas use pending pinto beans. reques...
1499997,5999974,55448,F,92750.90,1993-07-28,3-MEDIUM,Clerk#000000776,0,fts. requests affix furiously a
1499998,5999975,113398,F,63216.65,1993-07-25,1-URGENT,Clerk#000000813,0,oost! ironic instructions h


In [11]:
%%time
lineitem.merge(orders, left_on="l_orderkey", right_on="o_orderkey").pipe(
    lambda df: df.copy()[
        (df["l_shipdate"] < "1998-09-02")
        & (df["o_orderpriority"].isin(("1-URGENT", "2-HIGH")))
    ]
).groupby(["l_returnflag", "l_linestatus"]).agg(
    {
        "l_extendedprice": ["sum", "min", "max", "mean"],
        "l_quantity": ["sum", "min", "max", "mean"],
    }
)

CPU times: user 6.03 s, sys: 3.86 s, total: 9.89 s
Wall time: 10.3 s


l_extendedprice                                 \
                                      sum    min       max          mean   
l_returnflag l_linestatus                                                  
A            F               2.267720e+10  904.0  104949.5  38303.426664   
N            F               5.965302e+08  920.0  104049.0  38461.006847   
             O               4.473406e+10  901.0  104749.5  38267.695102   
R            F               2.268310e+10  906.0  104899.5  38240.140532   

                          l_quantity                     
                                 sum min max       mean  
l_returnflag l_linestatus                                
A            F              15123892   1  50  25.545346  
N            F                397729   1  50  25.643391  
             O              29826993   1  50  25.515466  
R            F              15126938   1  50  25.501645

Executando todas essas transformações demorou 10.3 segundos. Mas podemos ser mais eficientes. Por exemplo, poderíamos esquecer várias colunas já que no fim só estamos interessados em `l_returnflag`, `l_linestatus`, `l_extendedprice` e `l_quantity` (além das colunas que usamos para filtrar os dados). Também somos ineficientes quando temos de filtrar e agregar os valores (escaneamos a tabela duas vezes, quando poderíamos fazer isso de uma vez só). Além disso, poderíamos melhorar na implementação de paralelismo e do algoritmo.

Vamos então deixar pandas de lado e ver como podemos melhorar isso.

![](https://media.giphy.com/media/EPcvhM28ER9XW/giphy.gif)

## Dask

[Dask](https://docs.dask.org/en/latest/dataframe.html) provavelmente é a primeira solução que vem à cabeça. Dask basicamente cria várias tabelas em pandas e vai paralelizando as operações em cores diferentes. Também é mais eficiente por usar [avaliação preguiçosa](https://pt.wikipedia.org/wiki/Avalia%C3%A7%C3%A3o_pregui%C3%A7osa) - primeiro traçamos o que queremos fazer e depois mandamos o computador executar as transformações. Isso nos deixa otimizar a computação das transformações.

In [151]:
# collapse
# dask
dd_lineitem = dd.from_pandas(lineitem, npartitions=1)
dd_orders = dd.from_pandas(orders, npartitions=1)

In [152]:
%%timeit
dd_lineitem.merge(dd_orders, left_on="l_orderkey", right_on="o_orderkey").pipe(
    lambda df: df.copy()[
        (df["l_shipdate"] < "1998-09-02")
        & (df["o_orderpriority"].isin(("1-URGENT", "2-HIGH")))
    ]
).groupby(["l_returnflag", "l_linestatus"]).agg(
    {
        "l_extendedprice": ["sum", "min", "max", "mean"],
        "l_quantity": ["sum", "min", "max", "mean"],
    }
).compute()

CPU times: user 6.56 s, sys: 1.63 s, total: 8.2 s
Wall time: 8.23 s


l_extendedprice                                 \
                                      sum    min       max          mean   
l_returnflag l_linestatus                                                  
A            F               2.267720e+10  904.0  104949.5  38303.426664   
N            F               5.965302e+08  920.0  104049.0  38461.006847   
             O               4.473406e+10  901.0  104749.5  38267.695102   
R            F               2.268310e+10  906.0  104899.5  38240.140532   

                          l_quantity                     
                                 sum min max       mean  
l_returnflag l_linestatus                                
A            F              15123892   1  50  25.545346  
N            F                397729   1  50  25.643391  
             O              29826993   1  50  25.515466  
R            F              15126938   1  50  25.501645

## Polars

In [259]:
# collapse
# polars
pl_lineitem = pl.DataFrame(lineitem)
pl_orders = pl.DataFrame(orders)

In [263]:
%%timeit
pl_lineitem.join(pl_orders, left_on="l_orderkey", right_on="o_orderkey").filter(
    (pl.col("l_shipdate") < "1998-09-02")
    & (
        (pl.col("o_orderpriority") == "1-URGENT")
        | (pl.col("o_orderpriority") == "2-HIGH")
    )
).groupby(["l_returnflag", "l_linestatus"]).agg(
    {
        "l_extendedprice": ["sum", "min", "max", "mean"],
        "l_quantity": ["sum", "min", "max", "mean"],
    }
)

3.05 s ± 1.23 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [271]:
# hide
# using lazy eval
import time

pl_lineitem = pl_lineitem.lazy()
pl_orders = pl_orders.lazy()

In [275]:
# hide
start = time.monotonic()

# lazy dataframe
_df = (
    pl_lineitem.join(pl_orders, left_on="l_orderkey", right_on="o_orderkey").filter(
        (pl.col("l_shipdate") < "1998-09-02")
        & (
            (pl.col("o_orderpriority") == "1-URGENT")
            | (pl.col("o_orderpriority") == "2-HIGH")
        )
    )
    # could not delay collection - error (not finding `o_orderkey` column)
    .collect()
)
# eager dataframe
_df = _df.groupby(["l_returnflag", "l_linestatus"]).agg(
    [
        pl.sum("l_extendedprice"),
        pl.min("l_extendedprice"),
        pl.max("l_extendedprice"),
        pl.mean("l_extendedprice"),
        pl.sum("l_quantity"),
        pl.min("l_quantity"),
        pl.max("l_quantity"),
        pl.mean("l_quantity"),
    ],
)

print(f"Wall time: {time.monotonic() - start:.2f}s")
_df

Wall time: 5.74s


l_returnflag,l_linestatus,l_extendedprice_sum,l_extendedprice_min,l_extendedprice_max,l_extendedprice_mean,l_quantity_sum,l_quantity_min,l_quantity_max,l_quantity_mean
str,str,f64,f64,f64,f64,i64,i64,i64,f64
"""A""","""F""",2.2677199025699192e10,904,1.049495e5,3.830342666419926e4,15123892,1,50,25.54534567707304
"""N""","""F""",5.965302162000039e8,920,1.04049e5,3.8461006847195604e4,397729,1,50,25.643391360412636
"""N""","""O""",4.473405541680963e10,901,1.047495e5,3.8267695101622725e4,29826993,1,50,25.515466087014545
"""R""","""F""",2.2683095360319317e10,906,1.048995e5,3.824014053242183e4,15126938,1,50,25.501644539975555


## Datatable

In [179]:
# collapse
# datatable
dt_lineitem = dt.Frame(lineitem)
dt_orders = dt.Frame(orders)

In [180]:
%%timeit
# preparação
dt_lineitem.names = {"l_orderkey": "orderkey"}
dt_orders.names = {"o_orderkey": "orderkey"}

# dt_lineitem.key = "orderkey"  # não podemos usar como `key` existem valores repetidos
dt_orders.key = "orderkey"

# executando a query
dt_lineitem[
    (g.o_orderpriority == "1-URGENT") | (g.o_orderpriority == "2-HIGH"),
    :,
    join(dt_orders),
][
    :,
    {
        "sum(l_extendedprice)": dt.sum(f.l_extendedprice),
        "min(l_extendedprice)": dt.min(f.l_extendedprice),
        "max(l_extendedprice)": dt.max(f.l_extendedprice),
        "mean(l_extendedprice)": dt.mean(f.l_extendedprice),
        "sum(l_quantity)": dt.sum(f.l_quantity),
        "min(l_quantity)": dt.min(f.l_quantity),
        "max(l_quantity)": dt.max(f.l_quantity),
        "mean(l_quantity)": dt.mean(f.l_quantity),
    },
    by(f.l_returnflag, f.l_linestatus),
]

CPU times: user 6.79 s, sys: 271 ms, total: 7.06 s
Wall time: 1.17 s


,l_returnflag,l_linestatus,sum(l_extendedprice),min(l_extendedprice),max(l_extendedprice),mean(l_extendedprice),sum(l_quantity),min(l_quantity),max(l_quantity),mean(l_quantity)
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,A,F,2.26772e+10,904,104950,38303.4,15123892,1,50,25.5453
1,N,F,5.9653e+08,920,104049,38461,397729,1,50,25.6434
2,N,O,4.60456e+10,901,104750,38264.7,30703038,1,50,25.5147
3,R,F,2.26831e+10,906,104900,38240.1,15126938,1,50,25.5016


## PandaSQL

In [ ]:
!pip install pandasql

In [267]:
# hide_output
from pandasql import sqldf

start = time.monotonic()

_df = sqldf(
    """
SELECT l_returnflag,
       l_linestatus,
       SUM(l_extendedprice),
       MIN(l_extendedprice),
       MAX(l_extendedprice),
       AVG(l_extendedprice),
       SUM(l_quantity),
       MIN(l_quantity),
       MAX(l_quantity),
       AVG(l_quantity)
FROM lineitem AS lineitem
JOIN orders AS orders ON (l_orderkey=o_orderkey)
WHERE l_shipdate <= DATE '1998-09-02'
  AND o_orderpriority IN ('1-URGENT', '2-HIGH')
GROUP BY l_returnflag,
         l_linestatus
""",
    globals(),
)

print(f"Wall time: {time.monotonic() - start:.2f}s")
_df

PandaSQLException: (sqlite3.OperationalError) near "'1998-09-02'": syntax error
[SQL: 
SELECT l_returnflag,
       l_linestatus,
       SUM(l_extendedprice),
       MIN(l_extendedprice),
       MAX(l_extendedprice),
       AVG(l_extendedprice),
       SUM(l_quantity),
       MIN(l_quantity),
       MAX(l_quantity),
       AVG(l_quantity)
FROM lineitem AS lineitem
JOIN orders AS orders ON (l_orderkey=o_orderkey)
WHERE l_shipdate <= DATE '1998-09-02'
  AND o_orderpriority IN ('1-URGENT', '2-HIGH')
GROUP BY l_returnflag,
         l_linestatus
]
(Background on this error at: http://sqlalche.me/e/14/e3q8)

## DuckDB

In [5]:
lineitem = duckdb.query("SELECT * FROM 'lineitemsf1.snappy.parquet'").to_df()
orders = duckdb.query("SELECT * FROM 'orders.parquet'").to_df()

lineitem.head()

,l_orderkey,l_partkey,l_suppkey,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,l_shipdate,l_commitdate,l_receiptdate,l_shipinstruct,l_shipmode,l_comment
0,1,155190,7706,1,17,21168.23,0.04,0.02,N,O,1996-03-13,1996-02-12,1996-03-22,DELIVER IN PERSON,TRUCK,egular courts above the
1,1,67310,7311,2,36,45983.16,0.09,0.06,N,O,1996-04-12,1996-02-28,1996-04-20,TAKE BACK RETURN,MAIL,ly final dependencies: slyly bold
2,1,63700,3701,3,8,13309.60,0.10,0.02,N,O,1996-01-29,1996-03-05,1996-01-31,TAKE BACK RETURN,REG AIR,"riously. regular, express dep"
3,1,2132,4633,4,28,28955.64,0.09,0.06,N,O,1996-04-21,1996-03-30,1996-05-16,NONE,AIR,lites. fluffily even de
4,1,24027,1534,5,24,22824.48,0.10,0.04,N,O,1996-03-30,1996-03-14,1996-04-01,NONE,FOB,pending foxes. slyly re


In [262]:
%%timeit
duckdb.query(
    """
SELECT l_returnflag,
       l_linestatus,
       SUM(l_extendedprice),
       MIN(l_extendedprice),
       MAX(l_extendedprice),
       AVG(l_extendedprice),
       SUM(l_quantity),
       MIN(l_quantity),
       MAX(l_quantity),
       AVG(l_quantity)
FROM lineitem AS lineitem
JOIN orders AS orders ON (l_orderkey=o_orderkey)
WHERE l_shipdate <= DATE '1998-09-02'
  AND o_orderpriority IN ('1-URGENT', '2-HIGH')
GROUP BY l_returnflag,
         l_linestatus
"""
).to_df()

627 ms ± 25.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Mais info
<!-- - query opt: https://duckdb.org/2021/05/14/sql-on-pandas.html
- pandas trend: https://insights.stackoverflow.com/trends?tags=pandas%2Cnumpy%2Cdask%2Cspacy%2Ctensorflow%2Cpytorch%2Cscikit-learn%2Cpyspark%2Cpostgresql
- benchmark: https://h2oai.github.io/db-benchmark/
- duckdb caches stuff
- pandas-sql  -->

In [276]:
!rm lineitemsf1.snappy.parquet
!rm orders.parquet
!pip uninstall pandasql -y